# Lab 3

Author: Justin Ventura [[jventura3@gulls.salisbury.edu]]

Date: Thursday, September 17th, 2020.

## - Description -

This is where the fun begins >:)

### Import libraries which will be used and extract data:

In [124]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

a_frame = pd.read_csv("adult.data", header=None,
                      skipinitialspace=True, # this data has some extra whitespace
                      names=['age', 'workclass', 'fnlwgt', 'education',
                             'education-num', 'marital-status', 
                             'occupation', 'relationship', 'race',
                             'sex', 'capital_gain', 'capital_loss',
                             'hr_per_week', 'country', 'income'
                            ])
print(a_frame.shape)  # This outputs in form: (row, col).
a_frame.head()        # Print the first 5 entries.

(32561, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital_gain,capital_loss,hr_per_week,country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## 1) Pivoting the data.  

#### a) Rank occupation by most likely to make an income over $50k yearly.

In [134]:
# Create a sub-frame of the adult data with just the columns 'occupation' and 'income'.
occ_income = a_frame[['occupation', 'income']]

# Create two sub-frames of the previous sub-frame corresponding to two groups:
# Group 1: all of those who make over $50K, grouped by occupation, and
# Group 2: all of those who make under $50K, grouped by occupation.
greater = occ_income[occ_income['income'] == '>50K'].groupby('occupation').count()
lesser =  occ_income[occ_income['income'] == '<=50K'].groupby('occupation').count()

# Find the ratio for each occupation as a:b where a is the number of
# individuals who make over $50K, and b is the total number in that
# sample occupation.
totals = []
for pair in zip(greater['income'].tolist(), lesser['income'].tolist()):
    totals.append(sum(pair))

# This is simply the list of unique occupations in the dataframe.
occ_list = sorted(set(occ_income['occupation']))

# Just putting ratios next to their corresponding occupation
ratios = list(zip([pair[0]/pair[1] for pair in zip(greater['income'].tolist(), totals)], occ_list))

# Ignore the useless data (row of '?')
ratios.pop(0)

print("Occupations most to least likely to make over $50K annually based on sample:")
for elem in sorted(ratios, reverse=True):
    print('- ', elem[1], end='.\n')

Occupations most to least likely to make over $50K annually based on sample:
-  Exec-managerial.
-  Prof-specialty.
-  Protective-serv.
-  Tech-support.
-  Sales.
-  Craft-repair.
-  Transport-moving.
-  Adm-clerical.
-  Machine-op-inspct.
-  Farming-fishing.
-  Armed-Forces.
-  Handlers-cleaners.
-  Other-service.
-  Priv-house-serv.


### b) Calculate the median age of people who make more and less than $50k. Calculate the medians for each again, but now split apart by the sex of the adults.